### Assignment #4: Basic UI

DS4003 | Spring 2024

Objective: Practice buidling basic UI components in Dash. 

Task: Build an app that contains the following components user the gapminder dataset: `gdp_pcap.csv`. [Info](https://www.gapminder.org/gdp-per-capita/)

UI Components:
A dropdown menu that allows the user to select `country`
-   The dropdown should allow the user to select multiple countries
-   The options should populate from the dataset (not be hard-coded)
A slider that allows the user to select `year`
-   The slider should allow the user to select a range of years
-   The range should be from the minimum year in the dataset to the maximum year in the dataset
A graph that displays the `gdpPercap` for the selected countries over the selected years
-   The graph should display the gdpPercap for each country as a line
-   Each country should have a unique color
-   Graph DOES NOT need to interact with dropdown or slider
-   The graph should have a title and axis labels in reader friendly format  

Layout:  
- Use a stylesheet
- There should be a title at the top of the page
- There should be a description of the data and app below the title (3-5 sentences)
- The dropdown and slider should be side by side above the graph and take up the full width of the page
- The graph should be below the dropdown and slider and take up the full width of the page

Submission: 
- There should be only one app in your submitted work
- Comment your code
- Submit the html file of the notebook save as `DS4003_A4_LastName.html`


**For help you may use the web resources and pandas documentation. No co-pilot or ChatGPT.**

In [44]:
# import dependencies
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd

In [45]:
# load the dataset
df = pd.read_csv("gdp_pcap.csv")

In [46]:
# reshape dataframe 
melted_df = df.melt(id_vars=['country'], var_name = 'year', value_name = 'gdpPercap')
 # makes this so that there is a year and gdp calculated column

In [47]:
# initialize the dash app
app = dash.Dash(__name__)

In [48]:

# Define the layout of the app
app.layout = html.Div([
    html.H1("Gapminder GDP Per Capita Explorer"),
    html.P("Explore GDP per capita data from the Gapminder dataset. Use the slider to select the years. Use the drop down menu to select the countries you want to look at"),
    html.Div([
        # Dropdown menu for selecting countries
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in melted_df['country'].unique()],# make sure countries are not repeated 
            multi=True,
            value=['Afghanistan']  # Default selected countries, otherwise presents an error
        ),
        # Slider for selecting years
        dcc.RangeSlider(
            id='year-slider',
            min=int(melted_df['year'].min()), # minimum year value
            max=int(melted_df['year'].max()), # maximum year value 
            value=[int(melted_df['year'].min()), int(melted_df['year'].max())], # default
            marks={str(year): str(year) for year in melted_df['year'].unique()}, 
            step=None
        )
    ], style={'width': '100%', 'display': 'inline-block'}),
    
    # Graph for displaying GDP per capita, line 
    dcc.Graph(id='gdp-per-capita-graph')
]) 

# Define callback to update graph based on user inputs
@app.callback(
    Output('gdp-per-capita-graph', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('year-slider', 'value')]
)
def update_graph(selected_countries, selected_years):
    # Filter data based on selected countries and years
    filtered_df = melted_df[(melted_df['country'].isin(selected_countries)) & 
                            (melted_df['year'].astype(int).between(selected_years[0], selected_years[1]))]

    # Create traces for each selected country
    traces = []
    for country in selected_countries:
        country_data = filtered_df[filtered_df['country'] == country]
        traces.append(dict(
            x=country_data['year'],
            y=country_data['gdpPercap'],
            mode='lines',
            name=country
        ))

    # Define layout for the graph
    layout = dict(
        title='GDP Per Capita Over Time',
        xaxis={'title': 'Year'},
        yaxis={'title': 'GDP Per Capita'}
    )

    return {'data': traces, 'layout': layout}

if __name__ == '__main__':
    app.run_server(debug=True)
